In [2]:
# !python -m pip install corextopic
# !python -m pip install networkx

In [119]:
import matplotlib.pyplot as plt
import pandas as pd 
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from corextopic import corextopic as ct
from corextopic import vis_topic as vt # jupyter notebooks will complain matplotlib is being loaded twice
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import scipy.sparse as ss
%matplotlib inline
import nltk 
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

In [118]:
newline = []

textstring = ''
with open('Hansard_preCovid.csv', 'r', encoding='utf-16-le') as inp, open('newtextfile.txt', 'w') as out:
    for line in inp:
      newline.append(re.findall(r'"(.*)"', line))
      
temp = pd.Series(newline)

initialDF_preCovid = pd.DataFrame({'Text': temp})
clearWords = stopwords.words('english')
new_Stopwords = ['in', 'the', 'said', 'like','must', 'many', 'also', '(', ')', ',', '\'', ':', '.', '?', ']', '[', '-', 'mr.']
clearWords.extend(new_Stopwords)
lemma= WordNetLemmatizer()
table = str.maketrans('', '', string.punctuation)
initialDF_preCovid['Text'] = initialDF_preCovid['Text'].astype(str).str.lower()
initialDF_preCovid['Text'] = initialDF_preCovid['Text'].str.replace("|".join([r"^.*?speaker,", r"^.*?chair," ]), ' ', regex=True)
initialDF_preCovid['text_tokens'] = initialDF_preCovid['Text'].apply(word_tokenize)
initialDF_preCovid['text_tokens'] = initialDF_preCovid['text_tokens'].apply(lambda x: [item for item in x if item not in clearWords])
initialDF_preCovid['text_tokens'] = initialDF_preCovid['text_tokens'].apply(lambda x: [lemma.lemmatize(item) for item in x]) 
initialDF_preCovid['text_tokens'] = initialDF_preCovid['text_tokens'].apply(lambda w: [item.translate(table) for item in w])
# do we love me or what? isn't this join thing amazing?
# do we love me or what? isn't this join thing amazing?
# initialDF_preCovid['text_tokens'] = initialDF_preCovid['text_tokens'].apply(lambda x:' '.join([item for item in x if len(item)>3]))

In [120]:

# finding bigrams and trigrams mid-covid 

gram_wordlist = ['worker' , 'immigration',
                 'caregiver', 'canada', 'canadian', 'temporary', 'migrant', 'foreign', 'labour', 'committee', 'family', 'immigrant', 
                 'industry', 'farm','farmer', 'skill', 'skilled', 'bill', 'resident', 'farmer', 'agriculture', 'housing',
                   'employment', 'insurance' ,'employee' , 'harassment', 'residency' ,'residence', 'social', 'development', 'farming', 'program']
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_documents(initialDF_preCovid['text_tokens'])
finder.apply_freq_filter(15)
bigram_scores = finder.score_ngrams(bigram_measures.pmi)


trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = nltk.collocations.TrigramCollocationFinder.from_documents(initialDF_preCovid['text_tokens'])
finder.apply_freq_filter(15)
trigram_scores = finder.score_ngrams(trigram_measures.pmi)
bigram_pmi = pd.DataFrame(bigram_scores)
bigram_pmi.columns = ['bigram', 'pmi']
bigram_pmi.sort_values(by='pmi', axis = 0, ascending = True, inplace = True)
trigram_pmi = pd.DataFrame(trigram_scores)
trigram_pmi.columns = ['trigram', 'pmi']
trigram_pmi.sort_values(by='pmi', axis = 0, ascending = True, inplace = True)
# Filter for bigrams with only noun-type structures
def bigram_filter(bigram):
    tag = nltk.pos_tag(bigram)
    if tag[0][1] not in ['JJ', 'NN'] and tag[1][1] not in ['NN']:
        return False
    if bigram[0] in clearWords or bigram[1] in clearWords:
        return False
    if 'n' in bigram or 't' in bigram:
        return False
    if 'PRON' in bigram:
        return False
    return True
# Filter for trigrams with only noun-type structures
def trigram_filter(trigram):
    tag = nltk.pos_tag(trigram)
    if tag[0][1] not in ['JJ', 'NN'] and tag[1][1] not in ['JJ','NN']:
        return False
    if trigram[0] in clearWords or trigram[-1] in clearWords or trigram[1] in clearWords:
        return False
    if 'n' in trigram or 't' in trigram:
         return False
    if 'PRON' in trigram:
        return False
    return True 
# Can set pmi threshold to whatever makes sense - eyeball through and select threshold where n-grams stop making sense
# choose top 500 ngrams in this case ranked by PMI that have noun like structures
filtered_bigram = bigram_pmi[bigram_pmi.apply(lambda bigram:\
                                              bigram_filter(bigram['bigram'])\
                                              and bigram.pmi > 1, axis = 1)][:200]

filtered_trigram = trigram_pmi[trigram_pmi.apply(lambda trigram: \
                                                 trigram_filter(trigram['trigram'])\
                                                 and trigram.pmi >1, axis = 1)][:200]


bigrams = [' '.join(x) for x in filtered_bigram.bigram.values if x[0] in gram_wordlist or x[1] in gram_wordlist or x[-1] in gram_wordlist ]
trigrams = [' '.join(x) for x in filtered_trigram.trigram.values if x[0] in gram_wordlist or x[1] in gram_wordlist or x[-1] in gram_wordlist]

In [121]:
# Concatenate n-grams
def replace_ngram(x): 
    #first convert to string from list, then add the bigram and trigram strings. 
    for gram in trigrams:
        x = x.replace(gram, '_'.join(gram.split(' ')))
    for gram in bigrams:
        x = x.replace(gram, '_'.join(gram.split(' ')))
    return x

In [122]:
# Filter for only nouns adjectives and verbs
def noun_only(x):
    pos_comment = nltk.pos_tag(x)
    filtered = [word[0] for word in pos_comment if word[1] in ['NN', 'NNS', 'JJ','JJR', 'JJS', 'RB', 'RBR', 'RBS', 'VB', 'VBG', 'VBD', 'VBP']]
    # to filter both noun and verbs
    #filtered = [word[0] for word in pos_comment if word[1] in ['NN','VB', 'VBD', 'VBG','JJS','JJR', 'VBD', 'VBP', 'VBG', 'VBN', 'VBZ' 'VBN', 'VBZ']]
    return filtered

In [125]:
initialDF_preCovid.Text = initialDF_preCovid.Text.map(lambda x: replace_ngram(x))

#initialDF_midCovid['Text'] = initialDF_midCovid['Text'].apply(lambda w: [item.translate(table) for item in w])
initialDF_preCovid['Text'] = initialDF_preCovid['Text'].apply(word_tokenize)
initialDF_preCovid['Text'] = initialDF_preCovid['Text'].apply(lambda x: [item for item in x if item not in clearWords])
initialDF_preCovid['Text'] = initialDF_preCovid['Text'].apply(lambda x: [lemma.lemmatize(item) for item in x])
initialDF_preCovid['Text'] = initialDF_preCovid['Text'].apply(lambda x: [item for item in x if len(item) > 3])

In [127]:
final_text = initialDF_preCovid.Text.map(noun_only)

The topic model assumes input is in the form of a doc-word matrix, where rows are documents and columns are binary counts. We'll vectorize our data, take the top 20,000 words, and convert it to a sparse matrix to save on memory usage. Note, we use binary count vectors as input to the CorEx topic model.

In [156]:
vectorizer = CountVectorizer(stop_words= clearWords, max_features=8000, binary=True)
doc_word = vectorizer.fit_transform(final_text.astype(str))
doc_word = ss.csr_matrix(doc_word)
doc_word.shape # n_docs x m_words


c:\Users\Kriti Kapoor\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['mr'] not in stop_words.
  warnings.warn(


(305, 7444)

In [150]:
doc_word

<305x7444 sparse matrix of type '<class 'numpy.int64'>'
	with 43022 stored elements in Compressed Sparse Row format>

In [155]:
import numpy as np
# Get words that label the columns (needed to extract readable topics and make anchoring easier)
words = list(np.asarray(vectorizer.get_feature_names_out()))
not_digit_inds = [ind for ind,word in enumerate(words) if not word.isdigit()]
doc_word = doc_word[:,not_digit_inds]
words    = [word for ind,word in enumerate(words) if not word.isdigit()]

doc_word.shape # n_docs x m_words


IndexError: index (7443) out of range

In [152]:
# Train the CorEx topic model with 50 topics
topic_model = ct.Corex(n_hidden=21, words=words, max_iter=300, verbose=False, seed=11)
topic_model.fit(doc_word, words=words);


In [153]:
topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: part,important,support,first,well,community,place,take,economic,state
1: email,expectation,remain,obtain,tell,employee,stood,accept,de,profile
2: believe,seeking,operating,outcome,delivery,duration,document,constituency,refuse,finding
3: access,worker,riding,cultural,heritage,average,product,quarter,highest,contribution
4: received,currently,information,service,assistance,position,system,meet,class,international
5: challenge,mean,standard,interest,point,seek,company,drug,committee,mechanism
6: already,isds,protect,consult,wanted,green,evening,eu,threaten,covered
7: help,opportunity,immigrant,especially,form,newcomer,partner,focus,success,young
8: investor,even,question,export,nafta,generation,regulation,simply,took,compensation
9: provide,citizenship,office,meeting,provided,working,minister,accepted,common,report
10: individual,action,heard,rather,response,minimum,still,responsible,party,recently
11: intellectual,legal,territorial,discrimination,obligation,collective,rule,maintain,u

In [137]:
topic_model.get_topics(topic=3, n_words=100)

[('access', 0.1756294477457153, 1.0),
 ('worker', 0.14995232102039632, 1.0),
 ('riding', 0.1286332242542033, 1.0),
 ('cultural', 0.08850475212083216, 1.0),
 ('heritage', 0.08850475212083211, 1.0),
 ('average', 0.08465711353248635, 1.0),
 ('product', 0.0846571135324863, 1.0),
 ('quarter', 0.08042453339022203, 1.0),
 ('highest', 0.08030044291083906, 1.0),
 ('contribution', 0.07717172247810296, 1.0),
 ('join', 0.07359974446945079, 1.0),
 ('percentage', 0.07359974446945079, 1.0),
 ('political', 0.07348754065762092, 1.0),
 ('elsewhere', 0.07223358477962191, 1.0),
 ('present', 0.06591343358145012, 1.0),
 ('fiscal', 0.06541601140695334, 1.0),
 ('spent', 0.06470313526988974, 1.0),
 ('expenditure', 0.06340937669239494, 1.0),
 ('generous', 0.06340937669239494, 1.0),
 ('wage', 0.061683221230444875, 1.0),
 ('post', 0.05846747034894354, 1.0),
 ('administration', 0.057357938722852185, 1.0),
 ('length', 0.05735793872285214, 1.0),
 ('beneficial', 0.05502076033247172, 1.0),
 ('commission', 0.0550207603

In [154]:
anchor_words = [['temporary_foreign_worker', 'foreign_worker_program', 'foreign_worker', 'skilled'], ['livein_caregiver_program',  'caregiver', 'livein_caregiver']]
anchored_topic_modelt = ct.Corex(n_hidden=21, seed=11)
anchored_topic_modelt.fit(doc_word, words=words, anchors=anchor_words, anchor_strength=4)
for n in range(len(anchor_words)):
    topic_words,_,_ = zip(*anchored_topic_modelt.get_topics(topic=n))
    print('{} '.format(n) + ','.join(topic_words))


KeyboardInterrupt: 

In [143]:
anchored_topic_modelt.get_topics(topic=1, n_words=100)

[('worker', 0.47610059579992714, 1.0),
 ('support', 0.3420959786088661, 1.0),
 ('part', 0.33816743775790237, 1.0),
 ('important', 0.31594877919517333, 1.0),
 ('first', 0.30901666069694356, 1.0),
 ('community', 0.2990221786304724, 1.0),
 ('place', 0.28333462616836436, 1.0),
 ('canada', 0.27206822374367423, 1.0),
 ('well', 0.2688395770633176, 1.0),
 ('take', 0.2666929935123557, 1.0),
 ('economic', 0.25260773359759875, 1.0),
 ('country', 0.2480148079995145, 1.0),
 ('state', 0.2471557805380459, 1.0),
 ('opportunity', 0.24593572016228257, 1.0),
 ('life', 0.24572296154822804, 1.0),
 ('think', 0.236491930943929, 1.0),
 ('including', 0.2278585535687928, 1.0),
 ('cost', 0.22464482667507354, 1.0),
 ('issue', 0.2196815999410963, 1.0),
 ('canadian', 0.2193869981325237, 1.0),
 ('mean', 0.2193386793788577, 1.0),
 ('much', 0.21269054764041004, 1.0),
 ('want', 0.21226657261497514, 1.0),
 ('good', 0.20952545193881297, 1.0),
 ('right', 0.20594146464435484, 1.0),
 ('bill', 0.2035144938229316, 1.0),
 ('ti

mid covid

In [48]:
initialDF_midCovid= pd.read_csv('Hansard_mid_covid.csv', sep=',', encoding='utf-16-le')
initialDF_midCovid.drop(['Publication', 'First Name', 'Last Name','Constituency', 'Province', 'Date', 'Time', 'Page'], axis=1)
clearWords = stopwords.words('english')
new_Stopwords = ['in', 'the', 'said', 'like','must', 'many', 'also']
clearWords.extend(new_Stopwords)
lemma= WordNetLemmatizer()

initialDF_midCovid['Text'] = initialDF_midCovid['Text'].astype(str).str.lower()
initialDF_midCovid['Text'] = initialDF_midCovid['Text'].str.replace("|".join([r"^.*?speaker,", r"^.*?chair," ]), ' ', regex=True)
initialDF_midCovid['text_tokens'] = initialDF_midCovid['Text'].apply(word_tokenize)
initialDF_midCovid['text_tokens'] = initialDF_midCovid['text_tokens'].apply(lambda x: [item for item in x if item not in clearWords])
initialDF_midCovid['text_tokens'] = initialDF_midCovid['text_tokens'].apply(lambda x: [lemma.lemmatize(item) for item in x])
# do we love me or what? isn't this join thing amazing?
initialDF_midCovid['text_tokens'] = initialDF_midCovid['text_tokens'].apply(lambda x:' '.join([item for item in x if len(item)>3]))

In [49]:
vectorizer = CountVectorizer(stop_words='english', max_features=20000, binary=True)
doc_word = vectorizer.fit_transform(initialDF_midCovid['text_tokens'].astype(str))
doc_word = ss.csr_matrix(doc_word)
doc_word.shape # n_docs x m_words

# Get words that label the columns (needed to extract readable topics and make anchoring easier)
words = list(np.asarray(vectorizer.get_feature_names_out()))
not_digit_inds = [ind for ind,word in enumerate(words) if not word.isdigit()]
doc_word = doc_word[:,not_digit_inds]
words    = [word for ind,word in enumerate(words) if not word.isdigit()]


In [75]:
midcovid_unsup =  ct.Corex(n_hidden=60, words=words, max_iter=1200, verbose=False, seed=7)
midcovid_unsup.fit(doc_word, words=words);



In [ ]:
topics = midcovid_unsup.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

In [ ]:
midcovid_unsup.get_topics(topic=55, n_words=100)

In [87]:
anchor_words = [['workers', 'season', 'migrant']]
anchored_midCovidt= ct.Corex(n_hidden=45, seed=10)
anchored_midCovidt.fit(doc_word, words=words, anchors=anchor_words, anchor_strength=3)
for n in range(len(anchor_words)):
    topic_words,_,_ = zip(*anchored_midCovidt.get_topics(topic=n))
    print('{} '.format(n) + ','.join(topic_words))

0 migrant,workers,question,season,rising,proposes,floor,flexible,reminded,middle


In [ ]:
anchored_midCovidt.get_topics(topic =0, n_words=100)

In [100]:
from corextopic import vis_topic as vt
vt.vis_rep(topic_model, column_label=words, prefix='topic-model-example')

Print topics in text file


Hierarchical Modelling 

In [108]:
anchor_words = [['workers', 'season', 'migrant']]

new_mid_covid = ct.Corex(n_hidden=75)
new_mid_covid.fit(doc_word, words=words, anchors = anchor_words, anchor_strength=4)

ncm_layer2 = ct.Corex(n_hidden = 50)
ncm_layer2.fit(new_mid_covid.labels)    

ncm_layer3 = ct.Corex(n_hidden=20)
ncm_layer3.fit(ncm_layer2.labels)

ncm_layer4 = ct.Corex(n_hidden = 4)
ncm_layer4.fit(ncm_layer3.labels)


In [114]:
vt.vis_hierarchy([new_mid_covid, ncm_layer2, ncm_layer3, ncm_layer4],column_label=words, prefix='le' )

weight threshold is 0.210311 for graph with max of 100.000000 edges 
